<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase3/blob/main/Fase3_TechChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tech Challenge Fase 3

Vamos usar o modelo **BERT** no lugar de GPT-2, especificamente o modelo **`bert-base-uncased`** da Hugging Face, neste processo de fine-tuning muda um pouco, já que o BERT é um modelo "Masked Language Model" (MLM), e geralmente é utilizado para tarefas como classificação, predição de token, ou tarefas como QA (perguntas e respostas).

Vamos montar o notebook para usar o **BERT**. O objetivo é ajustar o modelo para responder a perguntas com base nas descrições dos produtos, o que pode ser tratado como uma tarefa de classificação de sequência (entrada e saída).


### 1. Instalar dependências

Primeira célula: Instala as bibliotecas necessárias.

In [ ]:
!pip install transformers datasets pandas

### 2. Carregar e ler o arquivo JSONL com pandas

Usando o pandas para carregar o arquivo `trn.json` no formato JSONL. Certifique-se de fazer upload do arquivo no Colab ou fornecer o caminho correto.


In [ ]:
import pandas as pd

# Carregar o arquivo trn.json
jsonl_file = '/content/trn.json'  # Ajuste o caminho conforme necessário

# Ler o arquivo jsonl com pandas
df = pd.read_json(jsonl_file, lines=True)

# Exibir uma amostra dos dados
df.head()

### 3. Selecionar as colunas relevantes

Aqui, selecionamos as colunas **"title"** (título do produto) e **"content"** (descrição do produto) para criar pares de entrada e saída.


In [ ]:
# Selecionar as colunas title e content
df = df[['title', 'content']]

# Exibir uma amostra dos dados filtrados
df.head()

### 4. Preparar os dados para o fine-tuning

Vamos criar pares de input-output para o modelo de BERT. Aqui, o input será a pergunta ("Descreva o produto com o título X?") e o output será a descrição do produto.


In [ ]:
# Função para criar prompts no formato correto
def create_prompt(row):
    prompt = f"Descreva o produto com o título '{row['title']}'?"
    return {"input_text": prompt, "output_text": row['content']}

# Aplicar a função a cada linha do dataframe
dataset = df.apply(create_prompt, axis=1)

# Converter para um DataFrame do Hugging Face
from datasets import Dataset
hf_dataset = Dataset.from_pandas(dataset)

# Exibir uma amostra do dataset final
hf_dataset[0]

### 5. Tokenização e preparação para o BERT

Como estamos usando o **BERT**, é importante garantir que os dados de entrada sejam tokenizados corretamente. Usaremos o tokenizador de **`bert-base-uncased`** e tokenizaremos tanto a pergunta quanto a resposta.

In [ ]:
from transformers import AutoTokenizer

# Carregar o tokenizador do BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Função para tokenizar a entrada e a saída
def tokenize_function(examples):
    return tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)

# Aplicar a tokenização ao dataset
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

# Exibir uma amostra dos dados tokenizados
tokenized_dataset[0]

### 6. Preparar o modelo BERT para o fine-tuning

Vamos carregar o modelo **`bert-base-uncased`** e adaptá-lo para uma tarefa de geração de texto. Para essa tarefa, usaremos a classe `AutoModelForSequenceClassification`, pois estamos tentando gerar uma saída baseada em uma entrada de sequência.

In [ ]:
from transformers import AutoModelForSequenceClassification

# Carregar o modelo BERT pré-treinado para classificação de sequência
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

### 7. Configurar o treinamento usando o Trainer da Hugging Face

Agora configuramos o treinamento. Ajustamos os argumentos de treinamento, como o número de épocas e o tamanho do batch.

In [ ]:
from transformers import Trainer, TrainingArguments

# Definir os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
)

# Configurar o Trainer para o BERT
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Executar o treinamento
trainer.train()

### 8. Avaliação do modelo após o fine-tuning

Após o treinamento, podemos testar o modelo para gerar uma descrição com base em um novo título fornecido pelo usuário.

In [ ]:
# Pergunta do usuário
user_input = "Descreva o produto com o título 'Headphones Bluetooth XYZ'?"

# Tokenizar a pergunta
inputs = tokenizer(user_input, return_tensors="pt")

# Gerar a resposta com o modelo fine-tunado
outputs = model(**inputs)

# Exibir o resultado
print("Saída:", outputs)

### 9. Conclusão e Salvamento do Modelo

Por fim, você pode salvar o modelo treinado para usá-lo posteriormente.

In [ ]:
# Salvar o modelo fine-tunado
model.save_pretrained("./fine_tuned_bert_model")
tokenizer.save_pretrained("./fine_tuned_bert_model")

print("Modelo treinado salvo!")